#📌 Extracão

In [1]:
import requests
import pandas as pd


In [2]:
url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/main/TelecomX_Data.json"
response = requests.get(url)
data = response.json()


In [3]:
df = pd.DataFrame(data)


In [4]:
print(df.shape)
print(df.head())


(7267, 6)
   customerID Churn                                           customer  \
0  0002-ORFBO    No  {'gender': 'Female', 'SeniorCitizen': 0, 'Part...   
1  0003-MKNFE    No  {'gender': 'Male', 'SeniorCitizen': 0, 'Partne...   
2  0004-TLHLJ   Yes  {'gender': 'Male', 'SeniorCitizen': 0, 'Partne...   
3  0011-IGKFF   Yes  {'gender': 'Male', 'SeniorCitizen': 1, 'Partne...   
4  0013-EXCHZ   Yes  {'gender': 'Female', 'SeniorCitizen': 1, 'Part...   

                                             phone  \
0   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   
1  {'PhoneService': 'Yes', 'MultipleLines': 'Yes'}   
2   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   
3   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   
4   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   

                                            internet  \
0  {'InternetService': 'DSL', 'OnlineSecurity': '...   
1  {'InternetService': 'DSL', 'OnlineSecurity': '...   
2  {'InternetService': 'Fiber optic', 'OnlineSecu...

#🔧 Transformação

In [5]:
# Verificar informações básicas
df.info()

# Visualizar as primeiras linhas para inspecionar a estrutura
df.head()

# Lista de colunas
print(df.columns.tolist())




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerID  7267 non-null   object
 1   Churn       7267 non-null   object
 2   customer    7267 non-null   object
 3   phone       7267 non-null   object
 4   internet    7267 non-null   object
 5   account     7267 non-null   object
dtypes: object(6)
memory usage: 340.8+ KB
['customerID', 'Churn', 'customer', 'phone', 'internet', 'account']


In [14]:
# URL do arquivo .md em modo RAW
dict_url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/main/TelecomX_dicionario.md"

# Requisição do conteúdo
response = requests.get(dict_url)

# Exibir o conteúdo do dicionário
print(response.text)


#### Dicionário de dados

* `customerID`: número de identificação único de cada cliente
* `Churn`: se o cliente deixou ou não a empresa 
* `gender`: gênero (masculino e feminino) 
* `SeniorCitizen`: informação sobre um cliente ter ou não idade igual ou maior que 65 anos 
* `Partner`:  se o cliente possui ou não um parceiro ou parceira
* `Dependents`: se o cliente possui ou não dependentes
* `tenure`:  meses de contrato do cliente
* `PhoneService`: assinatura de serviço telefônico 
* `MultipleLines`: assisnatura de mais de uma linha de telefone 
* `InternetService`: assinatura de um provedor internet 
* `OnlineSecurity`: assinatura adicional de segurança online 
* `OnlineBackup`: assinatura adicional de backup online 
* `DeviceProtection`: assinatura adicional de proteção no dispositivo 
* `TechSupport`: assinatura adicional de suporte técnico, menos tempo de espera
* `StreamingTV`: assinatura de TV a cabo 
* `StreamingMovies`: assinatura de streaming de filmes 
* `Contract`: tipo de co

In [16]:
import pandas as pd
import requests

# 1. Carregar dados brutos da API (JSON no GitHub)
url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/main/TelecomX_Data.json"
data = requests.get(url).json()
df = pd.DataFrame(data)

# 2. Normalizar colunas aninhadas
df_customer = pd.json_normalize(df['customer'])
df_phone    = pd.json_normalize(df['phone'])
df_internet = pd.json_normalize(df['internet'])
df_account  = pd.json_normalize(df['account'])

# 3. Concatenar tudo em um DataFrame único
df_clean = pd.concat(
    [df[['customerID', 'Churn']], df_customer, df_phone, df_internet, df_account],
    axis=1
)

# 4. Corrigir valores vazios em Churn
df_clean['Churn'] = df_clean['Churn'].replace('', 'Unknown')

# 5. Padronizar nomes de colunas (substituir pontos por underline)
df_clean.columns = df_clean.columns.str.replace('.', '_')

# 6. Converter valores numéricos
df_clean['Charges_Monthly'] = pd.to_numeric(df_clean['Charges_Monthly'], errors='coerce')
df_clean['Charges_Total']   = pd.to_numeric(df_clean['Charges_Total'], errors='coerce')

# 7. Verificar resultado
print(df_clean.info())
print(df_clean.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


In [17]:
# 1. Valores ausentes
print("Valores ausentes por coluna:\n", df_clean.isnull().sum())

# 2. Duplicatas no ID do cliente
duplicados = df_clean[df_clean.duplicated(subset="customerID", keep=False)]
print(f"\nNúmero de duplicatas: {duplicados.shape[0]}")

# 3. Valores únicos de Churn
print("\nValores únicos de Churn:", df_clean["Churn"].unique())

# 4. Tipos de dados
print("\nTipos de dados:")
print(df_clean.dtypes)

# 5. Conferir valores categóricos com possíveis erros
for col in df_clean.select_dtypes(include='object').columns:
    print(f"\nColuna '{col}': {df_clean[col].unique()[:10]}")  # Mostra até 10 valores únicos


Valores ausentes por coluna:
 customerID           0
Churn                0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
Charges_Monthly      0
Charges_Total       11
dtype: int64

Número de duplicatas: 0

Valores únicos de Churn: ['No' 'Yes' 'Unknown']

Tipos de dados:
customerID           object
Churn                object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport        

#📊 Carga e análise

#📄Relatorio Final